# Citypy introduction 

In [88]:
# You need to "pip install citipy" and then make sure you can import
from citipy import citipy
import random
import requests

In [89]:
# Create a list of coordinates. See this list comprehension method
# for creating them. 
coordinates = [[random.uniform(-90, 90) ,random.uniform(-180, 180)] for x in range(3)]
coordinates

[[-38.46205340035232, 164.1701344299285],
 [-80.6653474776954, 20.725378447257413],
 [43.590770641398706, 171.29711724676525]]

In [90]:
# Loop through the coordinates and find the nearest city
for coord in coordinates:
    print(f"{citipy.nearest_city(coord[0], coord[1]).city_name},{citipy.nearest_city(coord[0], coord[1]).country_code}")

te anau,nz
bredasdorp,za
nikolskoye,ru


In [91]:
#Module 6.2.2 explains how to get the api key and
# create config.py with the api key defined in it.
# I named mine weatherconfig.py.
# Remember to create a py file, not an ipynb file.
from weather_config import weather_api_key

In [92]:
# Current weather URL for the cities
# See https://openweathermap.org/current for documentation
for coord in coordinates:
    city_name_code = f"{citipy.nearest_city(coord[0], coord[1]).city_name},{citipy.nearest_city(coord[0], coord[1]).country_code}"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name_code}&units=Imperial&appid={weather_api_key}"
    #print(url)
    result = requests.get(url)
    #print(result.json())
    if result.json().get("cod") == 200:
        weather_data = result.json()
        temperature = weather_data["main"]["temp"]
        print(f"Temperature in {city_name_code} is {temperature} degrees farenheit")
    else:
        print(f"Failed to get the weather for {city_name_code}")

Temperature in te anau,nz is 37.42 degrees farenheit
Temperature in bredasdorp,za is 60.8 degrees farenheit
Temperature in nikolskoye,ru is 36.09 degrees farenheit
